## ML-第二版本-类别特征编码-时间序列特征
该版本在第一版的类别特征编码的基础上，增加时间序列特征

### 结果记录
+ 时间序列特征+lightgbm: 训练集AUC均值：0.9422837639748792, 训练集合AUC方差: 0.0030843473363093145
+ 时间序列特征+分箱+lightgbm: 训练集AUC均值：0.9949295080276108, 训练集合AUC方差: 0.0004933360925754542
+ 类别特征+时间序列特征+lightgbm: 训练集AUC均值：0.9459530611037537, 训练集合AUC方差: 0.0026239915479421365
+ 类别特征+时间序列特征WOE+lightgbm: 训练集AUC均值：0.9998999056269836, 训练集合AUC方差: 9.123546132262219e-06

### 结论
1. 时间序列特征+类别特征，由于不同维度的特征结合，使得数据的表达能力变强，对于同一模型来说，预测效果变得更好了
2. 时间序列特征+类别特征+分箱，效果有点过于好，使得我认为这部分内容可能有一点不真实，具体需要看这部分结果在oot上的表现如何，这部分效果太好我认为可能是因为woe编码时用了全体训练集合进行编码，相当于数据穿越了，但根据之前建模的经验，训练集测试集假设独立同分布，那么只要保证训练集于测试集的psi是稳定的就行，不过这里没有评估他们的psi是否是稳定的情况

## 导包

In [1]:
import os
import re
import numpy as np
import pandas as pd
import datetime
import pickle as pkl

import matplotlib.pyplot as plt
import seaborn as sns


import scorpyo as sp
from null_importance import get_null_importance
from time_sequence_feats import get_time_base, get_sequence_statis, get_sequence_groupby_statis
import category_encoders as ce

from gensim.models import word2vec



import time
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold

import warnings
warnings.filterwarnings("ignore")

pd.set_option('max_rows', 320, 'max_columns',100)

In [2]:
path_project = r'/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification'

# path dir
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results  = os.path.join(path_project, 'results')
path_results_version  = os.path.join(path_results, 'version_2')

# path row_data
path_train = os.path.join(path_row_data, 'train.csv')
path_test  = os.path.join(path_row_data, 'evaluation_public.csv')
path_sample_submission = os.path.join(path_row_data, 'submit_example.csv')

## results
path_output_report = os.path.join(path_results, '01_原始数据探察_20221014.xlsx')

y_label = "is_risk"


# new_train_data
path_new_train_cate = os.path.join(path_new_data, 'train_cate.pkl')
path_new_test_cate  = os.path.join(path_new_data, 'test_cate.pkl')

path_new_train_time_sequence = os.path.join(path_new_data, 'train_time_sequence.pkl')
path_new_test_time_sequence  = os.path.join(path_new_data, 'test_time_sequence.pkl')

path_new_train_time_sequence_woe = os.path.join(path_new_data, 'train_time_sequence_woe.pkl')
path_new_test_time_sequence_woe  = os.path.join(path_new_data, 'test_time_sequence_woe.pkl')

In [3]:
df_row_train = pd.read_csv(path_train)
df_row_val  = pd.read_csv(path_test)

## 工具函数

In [4]:
def save_pkl(data, path_data):
    """将数据保存为pkl"""
    with open(path_data, 'wb') as f:
        pkl.dump(data, f)
        
def read_pkl(path_data):
    """读取pkl格式数据"""
    with open(path_data, 'rb') as f:
        data = pkl.load(f)
    return data

## 类别特征编码

In [5]:
remove_cols = ['op_datetime', 'op_month']
cate_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
       'browser_version', 'browser', 'os_type', 'os_version',
       'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url']


ord_encoder = ce.WOEEncoder(cols=cate_cols)

df_row_val[y_label] = np.nan
df_train_cate = ord_encoder.fit_transform(df_row_train.drop(columns=remove_cols+[y_label]), df_row_train[y_label] )
df_val_cate  = ord_encoder.transform(df_row_val.drop(columns=remove_cols+[y_label]))


display(df_train_cate.head())
display(df_val_cate.head())

save_pkl(df_train_cate, path_new_train_cate)
save_pkl(df_val_cate, path_new_test_cate)


,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,ip_type,http_status_code,op_city,log_system_transform,url
0,0,0.399244,-0.041851,0.123852,0.355653,0.014496,0.014496,0.009044,0.012388,-0.011044,-0.298867,-0.100602,-0.246014,-0.246014
1,1,0.090044,0.035442,-0.076528,0.012419,0.060103,-0.009554,0.009044,-0.021990,-0.011044,-0.298867,-0.069165,-0.150937,-0.150937
2,2,0.142991,-0.041851,-0.098600,0.092439,-0.059518,-0.068291,0.009044,0.012388,-0.011044,-0.298867,-0.100602,0.530533,-0.206373
3,3,-0.011160,-0.041851,-0.255592,-0.085693,0.041217,0.041217,0.009044,0.012388,-0.011044,-0.298867,0.034066,-0.284901,-0.284901
4,4,0.232119,0.035442,-0.032103,0.145578,0.041217,0.041217,0.009044,0.012388,-0.011044,-0.298867,-0.100602,-0.150937,-0.150937


,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,ip_type,http_status_code,op_city,log_system_transform,url
0,0,-0.064648,-0.041851,-0.276959,-0.118715,-0.089269,-0.089269,-0.089269,-0.089269,-0.011044,-0.298867,-0.106403,-0.284901,-0.284901
1,1,0.065465,-0.041851,-0.100239,0.002945,0.041217,0.041217,0.009044,0.012388,-0.011044,-0.298867,-0.106403,-0.246014,-0.246014
2,2,-0.150053,-0.041851,-0.412955,-0.237064,-0.077409,-0.068291,0.009044,0.012388,-0.011044,-0.298867,0.034066,-0.284901,-0.284901
3,3,-0.208282,-0.041851,-0.406171,-0.222533,0.041217,0.041217,0.009044,0.012388,-0.011044,-0.298867,-0.100602,-0.255350,-0.255350
4,4,0.100432,-0.041851,-0.089002,0.072266,0.006565,-0.009554,0.009044,0.012388,-0.011044,-0.298867,-0.100602,-0.284901,-0.284901


## 时间序列特征衍生
参考自：[时序特征挖掘的奇技淫巧](https://www.6aiq.com/article/1594474995881)

In [6]:
df = pd.concat([df_row_train, df_row_val]).reset_index(drop=True)
df = df.sort_values(by='op_datetime')

# 按客户进行统计这次认证和上次认证的时间差
df['op_second'] = pd.to_datetime(df['op_datetime'])
df['op_second1'] = df.groupby('device_num_transform')['op_second'].shift(1)
df['op_second2'] = df.groupby('device_num_transform')['op_second'].shift(2)
df['op_diff_second1'] = (df['op_second'] - df['op_second1']).map(lambda x: x.total_seconds())
df['op_diff_second2'] = (df['op_second'] - df['op_second2']).map(lambda x: x.total_seconds())

# 系统层面的一段时间
df['system_op_second'] = pd.to_datetime(df['op_datetime'])
df['system_op_second1'] = df['system_op_second'].shift(1)
df['system_op_second2'] = df['system_op_second'].shift(2)
df['system_op_diff_second1'] = (df['system_op_second'] - df['system_op_second1']).map(lambda x: x.total_seconds())
df['system_op_diff_second2'] = (df['system_op_second'] - df['system_op_second2']).map(lambda x: x.total_seconds())


df = df.drop(columns=['op_second','op_second1', 'op_second2',
                      'system_op_second', 'system_op_second1','system_op_second2'])
df.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,op_datetime,ip_type,http_status_code,op_city,log_system_transform,url,op_month,is_risk,op_diff_second1,op_diff_second2,system_op_diff_second1,system_op_diff_second2
44477,44477,xiongkai3397,rd,6H1iPLgBB,GCgxrFb69up7,chrome_93,chrome,win,win10,2022-01-07 02:44:29,内网,200,深圳,nHrKgKdJ1Mzt,xxx.com/github,2022-01,1.0,NaN,NaN,NaN,NaN
45489,45489,zhengguiying7117,rd,0mjaEf4SB,8ftsXFm5I1Ej,safari_13,safari,macos,macos_big_sur_11,2022-01-07 02:54:32,内网,200,成都,nHrKgKdJ1Mzt,xxx.com/github,2022-01,1.0,NaN,NaN,603.0,NaN
45706,45706,yuanjun5870,hr,1Vk2kEa4X,W1Cstajd8x1s,firefox_78,firefox,win,win7,2022-01-07 03:00:56,内网,200,深圳,a5G25puBl9xj,hr.xxx.com/,2022-01,1.0,NaN,NaN,384.0,987.0
45901,45901,zhoutingting3694,rd,4Wj6uxLx3,H8NAVsdws95G,edge_93,edge,win,win10,2022-01-07 04:29:34,内网,200,杭州,nHrKgKdJ1Mzt,xxx.com/github,2022-01,1.0,NaN,NaN,5318.0,5702.0
43827,43827,yanglin6562,sales,eK12oQmm8,GnkVqPSy5nnl,ie_9,ie,win,win10,2022-01-07 05:17:44,内网,200,重庆,sW0whYIx8LFM,work.xxx.com/task,2022-01,1.0,NaN,NaN,2890.0,8208.0


In [7]:
cate_cols = ['ip_transform', 'user_name', 'device_num_transform', 'department', 'browser_version', 'browser', 'os_type','os_version',
              'ip_type','http_status_code', 'op_city', 'log_system_transform', 'url']

df = get_time_base(df, cols='op_datetime')
df = get_sequence_statis(df, col='system_op_diff_second1', n=5, freq=3 )
df = get_sequence_groupby_statis(df, col='system_op_diff_second1',cate_cols= cate_cols, n=5, freq=3)
df = get_sequence_statis(df, col='system_op_diff_second2', n=5, freq=3 )
df = get_sequence_groupby_statis(df, col='system_op_diff_second2',cate_cols= cate_cols, n=5, freq=3)
df.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,op_datetime,ip_type,http_status_code,op_city,log_system_transform,url,op_month,is_risk,op_diff_second1,op_diff_second2,system_op_diff_second1,system_op_diff_second2,op_datetime_year,op_datetime_month,op_datetime_day,op_datetime_hour,op_datetime_minute,op_datetime_second,op_datetime_quarter,op_datetime_dayofweek,op_datetime_is_year_start,op_datetime_is_month_start,op_datetime_is_month_end,op_datetime_second_sin,op_datetime_second_cos,op_datetime_minute_sin,op_datetime_minute_cos,op_datetime_hour_sin,op_datetime_hour_cos,op_datetime_day_sin,op_datetime_day_cos,op_datetime_dayofweek_sin,op_datetime_dayofweek_cos,op_datetime_month_sin,op_datetime_month_cos,avg_3_system_op_diff_second1,median_3_system_op_diff_second1,max_3_system_op_diff_second1,min_3_system_op_diff_second1,std_3_system_op_diff_second1,skew_3_system_op_diff_second1,...,kurt_log_system_transform_system_op_diff_second2_9,avg_log_system_transform_system_op_diff_second2_12,median_log_system_transform_system_op_diff_second2_12,max_log_system_transform_system_op_diff_second2_12,min_log_system_transform_system_op_diff_second2_12,std_log_system_transform_system_op_diff_second2_12,skew_log_system_transform_system_op_diff_second2_12,kurt_log_system_transform_system_op_diff_second2_12,avg_log_system_transform_system_op_diff_second2_15,median_log_system_transform_system_op_diff_second2_15,max_log_system_transform_system_op_diff_second2_15,min_log_system_transform_system_op_diff_second2_15,std_log_system_transform_system_op_diff_second2_15,skew_log_system_transform_system_op_diff_second2_15,kurt_log_system_transform_system_op_diff_second2_15,avg_url_system_op_diff_second2_3,median_url_system_op_diff_second2_3,max_url_system_op_diff_second2_3,min_url_system_op_diff_second2_3,std_url_system_op_diff_second2_3,skew_url_system_op_diff_second2_3,kurt_url_system_op_diff_second2_3,avg_url_system_op_diff_second2_6,median_url_system_op_diff_second2_6,max_url_system_op_diff_second2_6,min_url_system_op_diff_second2_6,std_url_system_op_diff_second2_6,skew_url_system_op_diff_second2_6,kurt_url_system_op_diff_second2_6,avg_url_system_op_diff_second2_9,median_url_system_op_diff_second2_9,max_url_system_op_diff_second2_9,min_url_system_op_diff_second2_9,std_url_system_op_diff_second2_9,skew_url_system_op_diff_second2_9,kurt_url_system_op_diff_second2_9,avg_url_system_op_diff_second2_12,median_url_system_op_diff_second2_12,max_url_system_op_diff_second2_12,min_url_system_op_diff_second2_12,std_url_system_op_diff_second2_12,skew_url_system_op_diff_second2_12,kurt_url_system_op_diff_second2_12,avg_url_system_op_diff_second2_15,median_url_system_op_diff_second2_15,max_url_system_op_diff_second2_15,min_url_system_op_diff_second2_15,std_url_system_op_diff_second2_15,skew_url_system_op_diff_second2_15,kurt_url_system_op_diff_second2_15
44477,44477,xiongkai3397,rd,6H1iPLgBB,GCgxrFb69up7,chrome_93,chrome,win,win10,2022-01-07 02:44:29,内网,200,深圳,nHrKgKdJ1Mzt,xxx.com/github,2022-01,1.0,NaN,NaN,NaN,NaN,2022,1,7,2,44,29,1,4,False,False,False,0.104528,-0.994522,-0.994522,-0.104528,0.500000,0.866025,0.988468,0.151428,-0.433884,-0.900969,0.5,0.866025,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45489,45489,zhengguiying7117,rd,0mjaEf4SB,8ftsXFm5I1Ej,safari_13,safari,macos,macos_big_sur_11,2022-01-07 02:54:32,内网,200,成都,nHrKgKdJ1Mzt,xxx.com/github,2022-01,1.0,NaN,NaN,603.0,NaN,2022,1,7,2,54,32,1,4,False,False,False,-0.207912,-0.978148,-0.587785,0.809017,0.500000,0.866025,0.988468,0.151428,-0.433884,-0.900969,0.5,0.866025,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [8]:
remove_cols = ['user_name', 'department', 'ip_transform', 'device_num_transform',
       'browser_version', 'browser', 'os_type', 'os_version', 'op_datetime',
       'ip_type', 'http_status_code', 'op_city', 'log_system_transform', 'url',
       'op_month']

df = df.drop(columns=remove_cols)

df_train_time_sequence = df[df[y_label].notna()].reset_index(drop=True)
df_val_time_sequence = df[df[y_label].isna()].reset_index(drop=True)


save_pkl(df_train_time_sequence.drop(columns=[y_label]), path_new_train_time_sequence)
save_pkl(df_val_time_sequence.drop(columns=[y_label]), path_new_test_time_sequence)

## modeling

### 时间序列特征+lightgbm

#### 获取建模使用的特征

In [9]:
remove_cols = ['id']
feats = df_train_time_sequence.columns
feats = feats.drop(remove_cols)
feats = feats.drop(y_label)

In [10]:
df_train = df_train_time_sequence.reset_index(drop=True)
df_test  = df_val_time_sequence.reset_index(drop=True)

#### lightgbm

In [11]:
# feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [12]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [13]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.936	valid_1's auc: 0.934455
[20]	training's auc: 0.943442	valid_1's auc: 0.938349
[30]	training's auc: 0.945944	valid_1's auc: 0.942365
[40]	training's auc: 0.950453	valid_1's auc: 0.939032
[50]	training's auc: 0.960835	valid_1's auc: 0.939325
[60]	training's auc: 0.969724	valid_1's auc: 0.940175
[70]	training's auc: 0.975062	valid_1's auc: 0.940737
[80]	training's auc: 0.983025	valid_1's auc: 0.941331
[90]	training's auc: 0.986808	valid_1's auc: 0.942929
[100]	training's auc: 0.989743	valid_1's auc: 0.941913
[110]	training's auc: 0.99135	valid_1's auc: 0.941318
[120]	training's auc: 0.992695	valid_1's auc: 0.941231
[130]	training's auc: 0.99365	valid_1's auc: 0.941127
[140]	training's auc: 0.994235	valid_1's auc: 0.940608
[150]	training's auc: 0.994828	valid_1's auc: 0.94095
[160]	training's auc: 0.995312	valid_1's auc: 0.941037
[170]	training's auc: 0.995674	valid_1's auc: 0.940352
[180

,name,importance
342,avg_ip_type_system_op_diff_second1_3,58496.628951
71,max_ip_transform_system_op_diff_second1_6,48554.789771
73,std_ip_transform_system_op_diff_second1_6,29289.182863
377,avg_http_status_code_system_op_diff_second1_3,12852.454029
27,avg_3_system_op_diff_second1,8691.267977
832,avg_ip_type_system_op_diff_second2_3,3559.440289
554,max_ip_transform_system_op_diff_second2_3,3374.670243
20,op_datetime_hour_cos,2926.831052
517,avg_3_system_op_diff_second2,2382.405901
3,system_op_diff_second2,2077.267576


训练集AUC均值：0.9422837639748792, 训练集合AUC方差: 0.0030843473363093145


### 时间序列特征+分箱+lightgbm

#### 分箱

In [14]:
cols = df_train_time_sequence.columns
cols = cols.drop(['id', y_label])

ord_encoder = ce.WOEEncoder(cols=cols)

df_train_time_sequence_woe = ord_encoder.fit_transform(df_train_time_sequence, df_train_time_sequence[y_label] )
df_val_time_sequence_woe  = ord_encoder.transform(df_val_time_sequence)


display(df_train_time_sequence_woe.head())
display(df_val_time_sequence_woe.head())


save_pkl(df_train_time_sequence_woe.drop(columns=[y_label]), path_new_train_time_sequence_woe)
save_pkl(df_val_time_sequence_woe.drop(columns=[y_label]), path_new_test_time_sequence_woe)

,id,is_risk,op_diff_second1,op_diff_second2,system_op_diff_second1,system_op_diff_second2,op_datetime_year,op_datetime_month,op_datetime_day,op_datetime_hour,op_datetime_minute,op_datetime_second,op_datetime_quarter,op_datetime_dayofweek,op_datetime_is_year_start,op_datetime_is_month_start,op_datetime_is_month_end,op_datetime_second_sin,op_datetime_second_cos,op_datetime_minute_sin,op_datetime_minute_cos,op_datetime_hour_sin,op_datetime_hour_cos,op_datetime_day_sin,op_datetime_day_cos,op_datetime_dayofweek_sin,op_datetime_dayofweek_cos,op_datetime_month_sin,op_datetime_month_cos,avg_3_system_op_diff_second1,median_3_system_op_diff_second1,max_3_system_op_diff_second1,min_3_system_op_diff_second1,std_3_system_op_diff_second1,skew_3_system_op_diff_second1,kurt_3_system_op_diff_second1,avg_6_system_op_diff_second1,median_6_system_op_diff_second1,max_6_system_op_diff_second1,min_6_system_op_diff_second1,std_6_system_op_diff_second1,skew_6_system_op_diff_second1,kurt_6_system_op_diff_second1,avg_9_system_op_diff_second1,median_9_system_op_diff_second1,max_9_system_op_diff_second1,min_9_system_op_diff_second1,std_9_system_op_diff_second1,skew_9_system_op_diff_second1,kurt_9_system_op_diff_second1,...,kurt_log_system_transform_system_op_diff_second2_9,avg_log_system_transform_system_op_diff_second2_12,median_log_system_transform_system_op_diff_second2_12,max_log_system_transform_system_op_diff_second2_12,min_log_system_transform_system_op_diff_second2_12,std_log_system_transform_system_op_diff_second2_12,skew_log_system_transform_system_op_diff_second2_12,kurt_log_system_transform_system_op_diff_second2_12,avg_log_system_transform_system_op_diff_second2_15,median_log_system_transform_system_op_diff_second2_15,max_log_system_transform_system_op_diff_second2_15,min_log_system_transform_system_op_diff_second2_15,std_log_system_transform_system_op_diff_second2_15,skew_log_system_transform_system_op_diff_second2_15,kurt_log_system_transform_system_op_diff_second2_15,avg_url_system_op_diff_second2_3,median_url_system_op_diff_second2_3,max_url_system_op_diff_second2_3,min_url_system_op_diff_second2_3,std_url_system_op_diff_second2_3,skew_url_system_op_diff_second2_3,kurt_url_system_op_diff_second2_3,avg_url_system_op_diff_second2_6,median_url_system_op_diff_second2_6,max_url_system_op_diff_second2_6,min_url_system_op_diff_second2_6,std_url_system_op_diff_second2_6,skew_url_system_op_diff_second2_6,kurt_url_system_op_diff_second2_6,avg_url_system_op_diff_second2_9,median_url_system_op_diff_second2_9,max_url_system_op_diff_second2_9,min_url_system_op_diff_second2_9,std_url_system_op_diff_second2_9,skew_url_system_op_diff_second2_9,kurt_url_system_op_diff_second2_9,avg_url_system_op_diff_second2_12,median_url_system_op_diff_second2_12,max_url_system_op_diff_second2_12,min_url_system_op_diff_second2_12,std_url_system_op_diff_second2_12,skew_url_system_op_diff_second2_12,kurt_url_system_op_diff_second2_12,avg_url_system_op_diff_second2_15,median_url_system_op_diff_second2_15,max_url_system_op_diff_second2_15,min_url_system_op_diff_second2_15,std_url_system_op_diff_second2_15,skew_url_system_op_diff_second2_15,kurt_url_system_op_diff_second2_15
0,44477,1.0,2.82611,2.255699,0.000000,2.745681,-0.000105,-0.0546,-0.160133,4.819152,0.074188,0.027464,0.003825,-0.137237,-0.000105,0.004134,-0.005843,0.039855,0.039097,0.012152,0.021101,-0.129382,4.810130,-0.160133,-0.188744,-0.137237,-0.122193,-0.0546,-0.0546,3.033363,3.033363,3.033363,3.033363,3.033363,1.244151,-0.000105,3.592979,3.592979,3.592979,3.592979,3.592979,3.305297,3.305297,3.949654,3.949654,3.949654,3.949654,3.949654,3.949654,3.949654,...,0.119382,0.119705,0.119705,0.119705,0.119705,0.119705,0.119705,0.119705,0.11482,0.11482,0.11482,0.11482,0.11482,0.11482,0.11482,1.513537,1.513537,1.513537,1.513537,1.513537,1.416664,-0.000105,0.887963,0.887963,0.887963,0.887963,0.887963,1.051085,1.051085,0.826088,0.826088,0.826088,0.826088,0.826088,0.826088,0.826088,0.697202,0.697202,0.697202,0.697202,0.697202

,id,is_risk,op_diff_second1,op_diff_second2,system_op_diff_second1,system_op_diff_second2,op_datetime_year,op_datetime_month,op_datetime_day,op_datetime_hour,op_datetime_minute,op_datetime_second,op_datetime_quarter,op_datetime_dayofweek,op_datetime_is_year_start,op_datetime_is_month_start,op_datetime_is_month_end,op_datetime_second_sin,op_datetime_second_cos,op_datetime_minute_sin,op_datetime_minute_cos,op_datetime_hour_sin,op_datetime_hour_cos,op_datetime_day_sin,op_datetime_day_cos,op_datetime_dayofweek_sin,op_datetime_dayofweek_cos,op_datetime_month_sin,op_datetime_month_cos,avg_3_system_op_diff_second1,median_3_system_op_diff_second1,max_3_system_op_diff_second1,min_3_system_op_diff_second1,std_3_system_op_diff_second1,skew_3_system_op_diff_second1,kurt_3_system_op_diff_second1,avg_6_system_op_diff_second1,median_6_system_op_diff_second1,max_6_system_op_diff_second1,min_6_system_op_diff_second1,std_6_system_op_diff_second1,skew_6_system_op_diff_second1,kurt_6_system_op_diff_second1,avg_9_system_op_diff_second1,median_9_system_op_diff_second1,max_9_system_op_diff_second1,min_9_system_op_diff_second1,std_9_system_op_diff_second1,skew_9_system_op_diff_second1,kurt_9_system_op_diff_second1,...,kurt_log_system_transform_system_op_diff_second2_9,avg_log_system_transform_system_op_diff_second2_12,median_log_system_transform_system_op_diff_second2_12,max_log_system_transform_system_op_diff_second2_12,min_log_system_transform_system_op_diff_second2_12,std_log_system_transform_system_op_diff_second2_12,skew_log_system_transform_system_op_diff_second2_12,kurt_log_system_transform_system_op_diff_second2_12,avg_log_system_transform_system_op_diff_second2_15,median_log_system_transform_system_op_diff_second2_15,max_log_system_transform_system_op_diff_second2_15,min_log_system_transform_system_op_diff_second2_15,std_log_system_transform_system_op_diff_second2_15,skew_log_system_transform_system_op_diff_second2_15,kurt_log_system_transform_system_op_diff_second2_15,avg_url_system_op_diff_second2_3,median_url_system_op_diff_second2_3,max_url_system_op_diff_second2_3,min_url_system_op_diff_second2_3,std_url_system_op_diff_second2_3,skew_url_system_op_diff_second2_3,kurt_url_system_op_diff_second2_3,avg_url_system_op_diff_second2_6,median_url_system_op_diff_second2_6,max_url_system_op_diff_second2_6,min_url_system_op_diff_second2_6,std_url_system_op_diff_second2_6,skew_url_system_op_diff_second2_6,kurt_url_system_op_diff_second2_6,avg_url_system_op_diff_second2_9,median_url_system_op_diff_second2_9,max_url_system_op_diff_second2_9,min_url_system_op_diff_second2_9,std_url_system_op_diff_second2_9,skew_url_system_op_diff_second2_9,kurt_url_system_op_diff_second2_9,avg_url_system_op_diff_second2_12,median_url_system_op_diff_second2_12,max_url_system_op_diff_second2_12,min_url_system_op_diff_second2_12,std_url_system_op_diff_second2_12,skew_url_system_op_diff_second2_12,kurt_url_system_op_diff_second2_12,avg_url_system_op_diff_second2_15,median_url_system_op_diff_second2_15,max_url_system_op_diff_second2_15,min_url_system_op_diff_second2_15,std_url_system_op_diff_second2_15,skew_url_system_op_diff_second2_15,kurt_url_system_op_diff_second2_15
0,25002,NaN,0.0,0.0,0.000000,0.0,-0.000105,0.0,-0.166861,4.911555,-0.128535,0.067067,-0.010462,1.155995,-0.000105,-0.166861,-0.005843,0.058827,0.059845,-0.117981,-0.063917,1.224615,4.911555,-0.166861,0.001154,1.155995,0.143365,-0.0546,0.0,0.0,1.647069,0.0,0.000000,0.0,0.0,-0.000105,0.0,0.0,3.438828,3.351817,0.0,0.0,0.0,0.0,2.745681,3.438828,3.033363,0.0,0.0,0.0,...,0.0,0.000000,0.000000,3.949654,2.745681,0.0,0.0,0.0,0.0,0.000000,3.949654,-0.230633,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,-0.000105,0.0,0.000000,0.0,3.033363,0.0,0.0,0.0,0.000000,0.000000,0.0,3.033363,0.0,0.0,0.0,0.000000,0.000000,3.949654,2.745681,0.0,0.0,0.0,0.0,0.000000,3.949654,-0.172090,0.0,0.0,0.0
1,23653,NaN,0.0,0.0,1.647069,0.0,-0.000105,0.0,-0.166861,4.911555,0.114251,0.013465,-0.010462,1.155995,-0.000105,-0.166861,-0.005843,0.0

#### 获取建模使用的特征

In [15]:
remove_cols = ['id']
feats = df_train_time_sequence_woe.columns
feats = feats.drop(remove_cols)
feats = feats.drop(y_label)

In [16]:
df_train = df_train_time_sequence_woe.reset_index(drop=True)
df_test  = df_val_time_sequence_woe.reset_index(drop=True)

#### lightgbm

In [17]:
# feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [20]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [21]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.969297	valid_1's auc: 0.966564
[20]	training's auc: 0.978646	valid_1's auc: 0.976248
[30]	training's auc: 0.984539	valid_1's auc: 0.980411
[40]	training's auc: 0.989127	valid_1's auc: 0.984201
[50]	training's auc: 0.991663	valid_1's auc: 0.986043
[60]	training's auc: 0.993496	valid_1's auc: 0.987686
[70]	training's auc: 0.99473	valid_1's auc: 0.98882
[80]	training's auc: 0.995581	valid_1's auc: 0.989993
[90]	training's auc: 0.996263	valid_1's auc: 0.990666
[100]	training's auc: 0.996717	valid_1's auc: 0.991204
[110]	training's auc: 0.997143	valid_1's auc: 0.991815
[120]	training's auc: 0.997468	valid_1's auc: 0.992125
[130]	training's auc: 0.997802	valid_1's auc: 0.992456
[140]	training's auc: 0.998079	valid_1's auc: 0.992754
[150]	training's auc: 0.998297	valid_1's auc: 0.99298
[160]	training's auc: 0.998495	valid_1's auc: 0.993098
[170]	training's auc: 0.998678	valid_1's auc: 0.993247
[

[100]	training's auc: 0.996811	valid_1's auc: 0.99208
[110]	training's auc: 0.997222	valid_1's auc: 0.992566
[120]	training's auc: 0.997561	valid_1's auc: 0.992819
[130]	training's auc: 0.997852	valid_1's auc: 0.993016
[140]	training's auc: 0.998114	valid_1's auc: 0.993261
[150]	training's auc: 0.998355	valid_1's auc: 0.993408
[160]	training's auc: 0.998566	valid_1's auc: 0.993514
[170]	training's auc: 0.998739	valid_1's auc: 0.993651
[180]	training's auc: 0.998911	valid_1's auc: 0.993753
[190]	training's auc: 0.999045	valid_1's auc: 0.993762
[200]	training's auc: 0.999159	valid_1's auc: 0.993816
[210]	training's auc: 0.999267	valid_1's auc: 0.993842
[220]	training's auc: 0.999367	valid_1's auc: 0.993886
[230]	training's auc: 0.999442	valid_1's auc: 0.993923
[240]	training's auc: 0.999522	valid_1's auc: 0.993957
[250]	training's auc: 0.999586	valid_1's auc: 0.993962
[260]	training's auc: 0.999648	valid_1's auc: 0.994012
[270]	training's auc: 0.999703	valid_1's auc: 0.994032
[280]	train

[490]	training's auc: 0.999999	valid_1's auc: 0.99461
[500]	training's auc: 1	valid_1's auc: 0.994603
[510]	training's auc: 1	valid_1's auc: 0.994609
[520]	training's auc: 1	valid_1's auc: 0.99464
[530]	training's auc: 1	valid_1's auc: 0.994616
[540]	training's auc: 1	valid_1's auc: 0.99463
[550]	training's auc: 1	valid_1's auc: 0.994659
[560]	training's auc: 1	valid_1's auc: 0.994667
[570]	training's auc: 1	valid_1's auc: 0.994689
[580]	training's auc: 1	valid_1's auc: 0.994691
[590]	training's auc: 1	valid_1's auc: 0.994723
[600]	training's auc: 1	valid_1's auc: 0.994722
[610]	training's auc: 1	valid_1's auc: 0.99474
[620]	training's auc: 1	valid_1's auc: 0.994737
[630]	training's auc: 1	valid_1's auc: 0.994751
[640]	training's auc: 1	valid_1's auc: 0.994755
[650]	training's auc: 1	valid_1's auc: 0.994761
[660]	training's auc: 1	valid_1's auc: 0.99478
[670]	training's auc: 1	valid_1's auc: 0.994779
[680]	training's auc: 1	valid_1's auc: 0.994772
[690]	training's auc: 1	valid_1's auc:

,name,importance
20,max_6_system_op_diff_second1,67960.359967
6,max_ip_transform_system_op_diff_second2_3,53038.844269
11,avg_ip_transform_system_op_diff_second2_3,34750.781476
12,avg_ip_transform_system_op_diff_second1_6,17531.766600
17,max_ip_type_system_op_diff_second1_6,13366.270780
14,op_diff_second1,9289.701179
42,op_diff_second2,6844.385117
35,avg_ip_type_system_op_diff_second2_9,6762.487266
49,avg_ip_type_system_op_diff_second2_6,5267.151633
16,std_ip_transform_system_op_diff_second2_3,5118.047789


训练集AUC均值：0.9949295080276108, 训练集合AUC方差: 0.0004933360925754542


### 类别特征+时间序列特征+lightgbm

#### 类别特征+时间序列特征

In [25]:
df_train_cate = read_pkl(path_new_train_cate)
df_val_cate = read_pkl(path_new_test_cate)

df_train_time_sequence = read_pkl(path_new_train_time_sequence)
df_val_time_sequence = read_pkl(path_new_test_time_sequence)

# df_train_time_sequence_woe = read_pkl(path_new_train_time_sequence_woe)
# df_val_time_sequence_woe = read_pkl(path_new_test_time_sequence_woe)

df_raw_train = pd.read_csv(path_train)
df_raw_val  = pd.read_csv(path_test)

In [26]:
df_row_train = pd.merge(left=df_train_cate, right=df_train_time_sequence, on='id')
df_row_val = pd.merge(left=df_val_cate, right=df_val_time_sequence, on='id')

df_row_train = pd.merge(left=df_row_train, right=df_raw_train[['id', y_label]], on='id')
df_row_val = pd.merge(left=df_row_val, right=df_raw_val[['id']], on='id')

In [27]:
df_row_train.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,ip_type,http_status_code,op_city,log_system_transform,url,op_diff_second1,op_diff_second2,system_op_diff_second1,system_op_diff_second2,op_datetime_year,op_datetime_month,op_datetime_day,op_datetime_hour,op_datetime_minute,op_datetime_second,op_datetime_quarter,op_datetime_dayofweek,op_datetime_is_year_start,op_datetime_is_month_start,op_datetime_is_month_end,op_datetime_second_sin,op_datetime_second_cos,op_datetime_minute_sin,op_datetime_minute_cos,op_datetime_hour_sin,op_datetime_hour_cos,op_datetime_day_sin,op_datetime_day_cos,op_datetime_dayofweek_sin,op_datetime_dayofweek_cos,op_datetime_month_sin,op_datetime_month_cos,avg_3_system_op_diff_second1,median_3_system_op_diff_second1,max_3_system_op_diff_second1,min_3_system_op_diff_second1,std_3_system_op_diff_second1,skew_3_system_op_diff_second1,kurt_3_system_op_diff_second1,avg_6_system_op_diff_second1,median_6_system_op_diff_second1,...,avg_log_system_transform_system_op_diff_second2_12,median_log_system_transform_system_op_diff_second2_12,max_log_system_transform_system_op_diff_second2_12,min_log_system_transform_system_op_diff_second2_12,std_log_system_transform_system_op_diff_second2_12,skew_log_system_transform_system_op_diff_second2_12,kurt_log_system_transform_system_op_diff_second2_12,avg_log_system_transform_system_op_diff_second2_15,median_log_system_transform_system_op_diff_second2_15,max_log_system_transform_system_op_diff_second2_15,min_log_system_transform_system_op_diff_second2_15,std_log_system_transform_system_op_diff_second2_15,skew_log_system_transform_system_op_diff_second2_15,kurt_log_system_transform_system_op_diff_second2_15,avg_url_system_op_diff_second2_3,median_url_system_op_diff_second2_3,max_url_system_op_diff_second2_3,min_url_system_op_diff_second2_3,std_url_system_op_diff_second2_3,skew_url_system_op_diff_second2_3,kurt_url_system_op_diff_second2_3,avg_url_system_op_diff_second2_6,median_url_system_op_diff_second2_6,max_url_system_op_diff_second2_6,min_url_system_op_diff_second2_6,std_url_system_op_diff_second2_6,skew_url_system_op_diff_second2_6,kurt_url_system_op_diff_second2_6,avg_url_system_op_diff_second2_9,median_url_system_op_diff_second2_9,max_url_system_op_diff_second2_9,min_url_system_op_diff_second2_9,std_url_system_op_diff_second2_9,skew_url_system_op_diff_second2_9,kurt_url_system_op_diff_second2_9,avg_url_system_op_diff_second2_12,median_url_system_op_diff_second2_12,max_url_system_op_diff_second2_12,min_url_system_op_diff_second2_12,std_url_system_op_diff_second2_12,skew_url_system_op_diff_second2_12,kurt_url_system_op_diff_second2_12,avg_url_system_op_diff_second2_15,median_url_system_op_diff_second2_15,max_url_system_op_diff_second2_15,min_url_system_op_diff_second2_15,std_url_system_op_diff_second2_15,skew_url_system_op_diff_second2_15,kurt_url_system_op_diff_second2_15,is_risk
0,0,0.399244,-0.041851,0.123852,0.355653,0.014496,0.014496,0.009044,0.012388,-0.011044,-0.298867,-0.100602,-0.246014,-0.246014,3418.0,11092.0,30.0,207.0,2022,1,18,19,10,41,1,1,False,False,False,-0.913545,-0.406737,0.866025,0.500000,-0.965926,0.258819,-0.485302,-0.874347,0.781831,0.623490,0.500000,0.866025,231.333333,177.0,487.0,30.0,233.294521,0.991186,NaN,142.500000,80.0,...,141.416667,154.5,215.0,40.0,55.662062,-0.382282,-0.909976,137.066667,154.0,215.0,40.0,54.626611,-0.335982,-0.960237,160.000000,166.0,207.0,107.0,50.269275,-0.529456,NaN,139.166667,134.0,207.0,87.0,44.750047,0.476213,-0.891429,125.888889,113.0,207.0,40.0,54.160512,-0.032093,-0.952888,141.416667,154.5,215.0,40.0,55.662062,-0.382282,-0.909976,137.066667,154.0,215.0,40.0,54.626611,-0.335982,-0.960237,0
1,1,0.090044,0.035442,-0.076528,0.012419,0.060103,-0.009554,0.009044,-0.021990,-0.011044,-0.298867,-0.069165,-0.150937,-0.150937,105393.0,111395.0,98.0,172.0,2022,4,1,17,4,0,2,4,False,True,False,0.000000,1.000000,0.406737,0.913545,-0.965926,-0.258819,0.201299,0.979530,-0.433884,-0.9009

#### 获取建模使用的特征

In [37]:
remove_cols = ['id']
feats = df_row_train.columns
feats = feats.drop(remove_cols)
feats = feats.drop(y_label)

In [38]:
df_train = df_row_train.reset_index(drop=True)
df_test  = df_row_val.reset_index(drop=True)

#### lightgbm

In [41]:
# feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [42]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [43]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.946987	valid_1's auc: 0.941647
[20]	training's auc: 0.94807	valid_1's auc: 0.942711
[30]	training's auc: 0.950491	valid_1's auc: 0.943375
[40]	training's auc: 0.953799	valid_1's auc: 0.941546
[50]	training's auc: 0.957278	valid_1's auc: 0.942406
[60]	training's auc: 0.96267	valid_1's auc: 0.941771
[70]	training's auc: 0.971291	valid_1's auc: 0.941209
[80]	training's auc: 0.97529	valid_1's auc: 0.941721
[90]	training's auc: 0.979057	valid_1's auc: 0.940563
[100]	training's auc: 0.981442	valid_1's auc: 0.939822
[110]	training's auc: 0.98373	valid_1's auc: 0.940521
[120]	training's auc: 0.985355	valid_1's auc: 0.939744
[130]	training's auc: 0.986738	valid_1's auc: 0.939478
Early stopping, best iteration is:
[31]	training's auc: 0.951133	valid_1's auc: 0.943516
----------- 1
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.942803	valid_1's auc: 0.937447
[20

,name,importance
1,max_ip_transform_system_op_diff_second1_6,53152.980467
0,avg_ip_type_system_op_diff_second1_3,51222.143152
3,avg_http_status_code_system_op_diff_second1_3,21582.169677
2,std_ip_transform_system_op_diff_second1_6,20891.208859
4,avg_3_system_op_diff_second1,13176.774422
7,max_ip_transform_system_op_diff_second2_3,5254.887168
13,avg_ip_transform_system_op_diff_second1_6,4665.806033
5,op_datetime_hour_cos,3694.868250
8,ip_transform,3179.524427
6,avg_ip_type_system_op_diff_second2_3,2588.830470


训练集AUC均值：0.9459530611037537, 训练集合AUC方差: 0.0026239915479421365


### 类别特征+时间序列特征WOE+lightgbm

#### 类别特征+时间序列特征

In [44]:
df_train_cate = read_pkl(path_new_train_cate)
df_val_cate = read_pkl(path_new_test_cate)

df_train_time_sequence_woe = read_pkl(path_new_train_time_sequence_woe)
df_val_time_sequence_woe = read_pkl(path_new_test_time_sequence_woe)

df_raw_train = pd.read_csv(path_train)
df_raw_val  = pd.read_csv(path_test)

In [45]:
df_row_train = pd.merge(left=df_train_cate, right=df_train_time_sequence_woe, on='id')
df_row_val = pd.merge(left=df_val_cate, right=df_val_time_sequence_woe, on='id')

df_row_train = pd.merge(left=df_row_train, right=df_raw_train[['id', y_label]], on='id')
df_row_val = pd.merge(left=df_row_val, right=df_raw_val[['id']], on='id')

In [46]:
df_row_train.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,ip_type,http_status_code,op_city,log_system_transform,url,op_diff_second1,op_diff_second2,system_op_diff_second1,system_op_diff_second2,op_datetime_year,op_datetime_month,op_datetime_day,op_datetime_hour,op_datetime_minute,op_datetime_second,op_datetime_quarter,op_datetime_dayofweek,op_datetime_is_year_start,op_datetime_is_month_start,op_datetime_is_month_end,op_datetime_second_sin,op_datetime_second_cos,op_datetime_minute_sin,op_datetime_minute_cos,op_datetime_hour_sin,op_datetime_hour_cos,op_datetime_day_sin,op_datetime_day_cos,op_datetime_dayofweek_sin,op_datetime_dayofweek_cos,op_datetime_month_sin,op_datetime_month_cos,avg_3_system_op_diff_second1,median_3_system_op_diff_second1,max_3_system_op_diff_second1,min_3_system_op_diff_second1,std_3_system_op_diff_second1,skew_3_system_op_diff_second1,kurt_3_system_op_diff_second1,avg_6_system_op_diff_second1,median_6_system_op_diff_second1,...,avg_log_system_transform_system_op_diff_second2_12,median_log_system_transform_system_op_diff_second2_12,max_log_system_transform_system_op_diff_second2_12,min_log_system_transform_system_op_diff_second2_12,std_log_system_transform_system_op_diff_second2_12,skew_log_system_transform_system_op_diff_second2_12,kurt_log_system_transform_system_op_diff_second2_12,avg_log_system_transform_system_op_diff_second2_15,median_log_system_transform_system_op_diff_second2_15,max_log_system_transform_system_op_diff_second2_15,min_log_system_transform_system_op_diff_second2_15,std_log_system_transform_system_op_diff_second2_15,skew_log_system_transform_system_op_diff_second2_15,kurt_log_system_transform_system_op_diff_second2_15,avg_url_system_op_diff_second2_3,median_url_system_op_diff_second2_3,max_url_system_op_diff_second2_3,min_url_system_op_diff_second2_3,std_url_system_op_diff_second2_3,skew_url_system_op_diff_second2_3,kurt_url_system_op_diff_second2_3,avg_url_system_op_diff_second2_6,median_url_system_op_diff_second2_6,max_url_system_op_diff_second2_6,min_url_system_op_diff_second2_6,std_url_system_op_diff_second2_6,skew_url_system_op_diff_second2_6,kurt_url_system_op_diff_second2_6,avg_url_system_op_diff_second2_9,median_url_system_op_diff_second2_9,max_url_system_op_diff_second2_9,min_url_system_op_diff_second2_9,std_url_system_op_diff_second2_9,skew_url_system_op_diff_second2_9,kurt_url_system_op_diff_second2_9,avg_url_system_op_diff_second2_12,median_url_system_op_diff_second2_12,max_url_system_op_diff_second2_12,min_url_system_op_diff_second2_12,std_url_system_op_diff_second2_12,skew_url_system_op_diff_second2_12,kurt_url_system_op_diff_second2_12,avg_url_system_op_diff_second2_15,median_url_system_op_diff_second2_15,max_url_system_op_diff_second2_15,min_url_system_op_diff_second2_15,std_url_system_op_diff_second2_15,skew_url_system_op_diff_second2_15,kurt_url_system_op_diff_second2_15,is_risk
0,0,0.399244,-0.041851,0.123852,0.355653,0.014496,0.014496,0.009044,0.012388,-0.011044,-0.298867,-0.100602,-0.246014,-0.246014,0.000000,0.548456,-0.959318,-0.590978,-0.000105,-0.054600,-0.301408,-0.055677,0.069334,0.067936,0.003825,-0.157232,-0.000105,0.004134,-0.005843,0.030055,0.036896,0.142991,0.011941,-0.285013,0.323980,-0.301408,-0.143343,-0.157232,0.143365,-0.054600,-0.054600,1.359386,-0.917881,-0.432373,-0.815147,0.0,0.0,-0.000105,0.666239,-0.582946,...,0.037631,-0.983020,0.238301,-0.264650,0.0,0.0,0.0,-0.704307,-0.744312,0.483918,-0.272030,0.0,0.0,0.0,-0.319044,-0.868610,-0.886628,-0.447877,0.0,0.0,-0.000105,-0.604223,-0.688306,-1.141024,-0.463145,0.0,0.0,0.0,-0.809667,-0.519025,-0.917881,-0.226442,0.0,0.0,0.0,-0.144691,-1.013191,0.308783,-0.262474,0.0,0.0,0.0,-0.473195,-0.534889,0.307294,-0.216744,0.0,0.0,0.0,0
1,1,0.090044,0.035442,-0.076528,0.012419,0.060103,-0.009554,0.009044,-0.021990,-0.011044,-0.298867,-0.069165,-0.150937,-0.150937,0.000000,0.000000,-1.115049,-0.861369,-0.000105,-0.010462,-0.166861,-0.425748,0.059071,0.043530,-0.010462,

#### 获取建模使用的特征

In [47]:
remove_cols = ['id']
feats = df_row_train.columns
feats = feats.drop(remove_cols)
feats = feats.drop(y_label)

In [48]:
df_train = df_row_train.reset_index(drop=True)
df_test  = df_row_val.reset_index(drop=True)

#### lightgbm

In [49]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [50]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.992351	valid_1's auc: 0.990632
[20]	training's auc: 0.995111	valid_1's auc: 0.992908
[30]	training's auc: 0.996765	valid_1's auc: 0.993898
[40]	training's auc: 0.997569	valid_1's auc: 0.995089
[50]	training's auc: 0.998185	valid_1's auc: 0.995743
[60]	training's auc: 0.998653	valid_1's auc: 0.996457
[70]	training's auc: 0.999059	valid_1's auc: 0.997018
[80]	training's auc: 0.999431	valid_1's auc: 0.997548
[90]	training's auc: 0.999651	valid_1's auc: 0.998042
[100]	training's auc: 0.999811	valid_1's auc: 0.998415
[110]	training's auc: 0.999901	valid_1's auc: 0.998691
[120]	training's auc: 0.999956	valid_1's auc: 0.998921
[130]	training's auc: 0.999982	valid_1's auc: 0.999109
[140]	training's auc: 0.999994	valid_1's auc: 0.999235
[150]	training's auc: 0.999999	valid_1's auc: 0.99935
[160]	training's auc: 1	valid_1's auc: 0.999421
[170]	training's auc: 1	valid_1's auc: 0.999483
[180]	trainin

[150]	training's auc: 0.999999	valid_1's auc: 0.999294
[160]	training's auc: 1	valid_1's auc: 0.999363
[170]	training's auc: 1	valid_1's auc: 0.999436
[180]	training's auc: 1	valid_1's auc: 0.999484
[190]	training's auc: 1	valid_1's auc: 0.99952
[200]	training's auc: 1	valid_1's auc: 0.999568
[210]	training's auc: 1	valid_1's auc: 0.999591
[220]	training's auc: 1	valid_1's auc: 0.999615
[230]	training's auc: 1	valid_1's auc: 0.999642
[240]	training's auc: 1	valid_1's auc: 0.999662
[250]	training's auc: 1	valid_1's auc: 0.999679
[260]	training's auc: 1	valid_1's auc: 0.999696
[270]	training's auc: 1	valid_1's auc: 0.999705
[280]	training's auc: 1	valid_1's auc: 0.999721
[290]	training's auc: 1	valid_1's auc: 0.999732
[300]	training's auc: 1	valid_1's auc: 0.999734
[310]	training's auc: 1	valid_1's auc: 0.999743
[320]	training's auc: 1	valid_1's auc: 0.999757
[330]	training's auc: 1	valid_1's auc: 0.999771
[340]	training's auc: 1	valid_1's auc: 0.999778
[350]	training's auc: 1	valid_1's 

[200]	training's auc: 1	valid_1's auc: 0.999564
[210]	training's auc: 1	valid_1's auc: 0.999591
[220]	training's auc: 1	valid_1's auc: 0.999613
[230]	training's auc: 1	valid_1's auc: 0.999627
[240]	training's auc: 1	valid_1's auc: 0.999649
[250]	training's auc: 1	valid_1's auc: 0.999673
[260]	training's auc: 1	valid_1's auc: 0.999692
[270]	training's auc: 1	valid_1's auc: 0.999705
[280]	training's auc: 1	valid_1's auc: 0.99972
[290]	training's auc: 1	valid_1's auc: 0.999729
[300]	training's auc: 1	valid_1's auc: 0.999736
[310]	training's auc: 1	valid_1's auc: 0.999749
[320]	training's auc: 1	valid_1's auc: 0.999754
[330]	training's auc: 1	valid_1's auc: 0.999766
[340]	training's auc: 1	valid_1's auc: 0.999777
[350]	training's auc: 1	valid_1's auc: 0.999785
[360]	training's auc: 1	valid_1's auc: 0.999791
[370]	training's auc: 1	valid_1's auc: 0.999796
[380]	training's auc: 1	valid_1's auc: 0.999798
[390]	training's auc: 1	valid_1's auc: 0.999803
[400]	training's auc: 1	valid_1's auc: 0.

[940]	training's auc: 1	valid_1's auc: 0.999895
[950]	training's auc: 1	valid_1's auc: 0.999895
[960]	training's auc: 1	valid_1's auc: 0.999894
[970]	training's auc: 1	valid_1's auc: 0.999895
[980]	training's auc: 1	valid_1's auc: 0.999895
[990]	training's auc: 1	valid_1's auc: 0.999896
[1000]	training's auc: 1	valid_1's auc: 0.999896
[1010]	training's auc: 1	valid_1's auc: 0.999896
[1020]	training's auc: 1	valid_1's auc: 0.999897
[1030]	training's auc: 1	valid_1's auc: 0.999897
[1040]	training's auc: 1	valid_1's auc: 0.999898
[1050]	training's auc: 1	valid_1's auc: 0.999898
[1060]	training's auc: 1	valid_1's auc: 0.999898
[1070]	training's auc: 1	valid_1's auc: 0.999897
[1080]	training's auc: 1	valid_1's auc: 0.999899
[1090]	training's auc: 1	valid_1's auc: 0.999899
[1100]	training's auc: 1	valid_1's auc: 0.999899
[1110]	training's auc: 1	valid_1's auc: 0.9999
[1120]	training's auc: 1	valid_1's auc: 0.9999
[1130]	training's auc: 1	valid_1's auc: 0.999901
[1140]	training's auc: 1	valid

[990]	training's auc: 1	valid_1's auc: 0.999899
[1000]	training's auc: 1	valid_1's auc: 0.999899
[1010]	training's auc: 1	valid_1's auc: 0.9999
[1020]	training's auc: 1	valid_1's auc: 0.9999
[1030]	training's auc: 1	valid_1's auc: 0.999899
[1040]	training's auc: 1	valid_1's auc: 0.999899
[1050]	training's auc: 1	valid_1's auc: 0.9999
[1060]	training's auc: 1	valid_1's auc: 0.9999
[1070]	training's auc: 1	valid_1's auc: 0.999901
[1080]	training's auc: 1	valid_1's auc: 0.999902
[1090]	training's auc: 1	valid_1's auc: 0.999901
[1100]	training's auc: 1	valid_1's auc: 0.999902
[1110]	training's auc: 1	valid_1's auc: 0.999902
[1120]	training's auc: 1	valid_1's auc: 0.999902
[1130]	training's auc: 1	valid_1's auc: 0.999902
[1140]	training's auc: 1	valid_1's auc: 0.999902
[1150]	training's auc: 1	valid_1's auc: 0.999903
[1160]	training's auc: 1	valid_1's auc: 0.999903
[1170]	training's auc: 1	valid_1's auc: 0.999903
[1180]	training's auc: 1	valid_1's auc: 0.999903
[1190]	training's auc: 1	vali

,name,importance
49,max_6_system_op_diff_second1,71834.593789
567,max_ip_transform_system_op_diff_second2_3,41744.029170
565,avg_ip_transform_system_op_diff_second2_3,29601.126655
75,avg_ip_transform_system_op_diff_second1_3,9432.886327
628,avg_user_name_system_op_diff_second2_15,7891.160254
593,avg_ip_transform_system_op_diff_second2_15,6898.400215
572,avg_ip_transform_system_op_diff_second2_6,6808.958639
364,max_ip_type_system_op_diff_second1_6,6288.210520
908,avg_http_status_code_system_op_diff_second2_15,4947.291076
586,avg_ip_transform_system_op_diff_second2_12,4908.466780


训练集AUC均值：0.9998999056269836, 训练集合AUC方差: 9.123546132262219e-06
